# Hydrated Docking with AutoDock Vina

This notebook implements hydrated docking methodology for fragment-sized ligands using explicit water molecules. The hydrated docking approach models bridging water molecules that may remain bound during ligand binding, improving prediction accuracy especially for fragment-based drug discovery.

**Key Features:**
- Explicit water molecule modeling using W atoms
- AutoDock4 forcefield for calibrated water interactions
- Water affinity map generation (`mapwater.py`)
- Post-processing with water scoring and filtering (`dry.py`)

**References:**
- Forli, S., & Olson, A. J. (2012). A force field with discrete displaceable waters and desolvation entropy for hydrated ligand docking. *J. Med. Chem.*, 55(2), 623-638.
- Tutorial: https://autodock-vina.readthedocs.io/en/latest/docking_hydrated.html

## Setup and Configuration

**Important:** Skip the condacolab cells if running in Docker environment (cells 1-2 from Colab versions).

Configure your experiment parameters below:

In [ ]:
# Import Modules & Locate Command Line Scripts

# Import modules
from os.path import exists
import sys, platform, os, subprocess
from prody import *
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit, py3Dmol
print("rdkit version:", rdkit.__version__)
print("py3Dmol version:", py3Dmol.__version__)
from ipywidgets import interact, IntSlider
import ipywidgets, copy
from IPython.display import display, Markdown
import datetime

# Helper functions
def locate_file(from_path = None, query_path = None, query_name = "query file"):
    if not from_path or not query_path:
        raise ValueError("Must specify from_path and query_path")

    possible_path = list(from_path.glob(query_path))

    if not possible_path:
        raise FileNotFoundError(f"Cannot find {query_name} from {from_path} by {query_path}")

    return_which = (
        f"using {query_name} at:\n"
        f"{possible_path[0]}\n"
    )
    print(return_which)

    return possible_path[0]

# Command-line scripts
scrub = locate_file(from_path = Path("/opt/conda/envs/vina/bin"), query_path = "scrub.py", query_name = "scrub.py")
mk_prepare_ligand = locate_file(from_path = Path("/opt/conda/envs/vina/bin"), query_path = "mk_prepare_ligand.py", query_name = "mk_prepare_ligand.py")
mk_prepare_receptor = locate_file(from_path = Path("/opt/conda/envs/vina/bin"), query_path = "mk_prepare_receptor.py", query_name = "mk_prepare_receptor.py")
mk_export = locate_file(from_path = Path("/opt/conda/envs/vina/bin"), query_path = "mk_export.py", query_name = "mk_export.py")

# Hydrated docking specific scripts
mapwater = locate_file(from_path = Path("/opt/conda/envs/vina/bin"), query_path = "mapwater.py", query_name = "mapwater.py")
dry = locate_file(from_path = Path("/opt/conda/envs/vina/bin"), query_path = "dry.py", query_name = "dry.py")

# Locate reduce2 in conda install prefix
full_py_version = platform.python_version()
major_and_minor = ".".join(full_py_version.split(".")[:2])
env_path = Path("/opt/conda/envs/vina")
reduce2_path = f"lib/python{major_and_minor}/site-packages/mmtbx/command_line/reduce2.py"
reduce2 = locate_file(from_path = env_path, query_path = reduce2_path, query_name = "reduce2.py")

# Locate geostd in current path
geostd_path = locate_file(from_path = Path.cwd(), query_path = "geostd", query_name = "geostd")

print("\n=== Hydrated Docking Tools Check ===")
print(f"mapwater.py: {mapwater}")
print(f"dry.py: {dry}")
print("\n✓ All hydrated docking tools located successfully!")

## Experiment Configuration

Set up your experiment parameters:

In [ ]:
# Experiment Configuration
experiment_date = datetime.date.today().strftime("%Y-%m-%d")
pdb_id = "1uw6"  # Example: acetylcholine binding protein
target_name = "AChBP"  # Target protein name
ligand_name = "nicotine"  # Ligand name for this experiment

# File paths
receptor_pdb = f"{pdb_id}_receptorH.pdb"  # Input receptor with hydrogens
ligand_sdf = f"{pdb_id}_ligand.sdf"  # Input ligand SDF file

# Results directory
results_dir = f"results/{experiment_date}_Hydrated_{pdb_id.upper()}_{ligand_name}-{target_name}"
os.makedirs(results_dir, exist_ok=True)

print(f"Experiment: {experiment_date}_Hydrated_{pdb_id.upper()}_{ligand_name}-{target_name}")
print(f"Results directory: {results_dir}")

# Docking box configuration (modify according to your binding site)
box_center = [83.640, 69.684, -10.124]  # x, y, z coordinates
box_size = [15, 15, 15]  # x, y, z dimensions

print(f"\nDocking box:")
print(f"  Center: {box_center}")
print(f"  Size: {box_size}")

## 1. Receptor Preparation for Hydrated Docking

Prepare the receptor with hydrogen addition and generate the Grid Parameter File (GPF) required for AutoGrid4 map generation:

In [ ]:
# 1. Receptor Preparation with GPF generation

receptor_base = f"{pdb_id}_receptor"
receptor_output = f"{results_dir}/{receptor_base}"

# Prepare receptor with GPF generation (-g flag)
receptor_cmd = [
    str(mk_prepare_receptor),
    "-i", receptor_pdb,
    "-o", receptor_output,
    "-p",  # Add polar hydrogens
    "-g",  # Generate GPF file for AutoGrid4
    "--box_center", str(box_center[0]), str(box_center[1]), str(box_center[2]),
    "--box_size", str(box_size[0]), str(box_size[1]), str(box_size[2])
]

print("Preparing receptor with GPF generation...")
print(f"Command: {' '.join(receptor_cmd)}")

result = subprocess.run(receptor_cmd, capture_output=True, text=True)
print("\nSTDOUT:")
print(result.stdout)
if result.stderr:
    print("\nSTDERR:")
    print(result.stderr)

# Check generated files
receptor_pdbqt = f"{receptor_output}.pdbqt"
receptor_gpf = f"{receptor_output}.gpf"

print(f"\nGenerated files:")
print(f"  Receptor PDBQT: {'✓' if os.path.exists(receptor_pdbqt) else '✗'} {receptor_pdbqt}")
print(f"  Grid Parameter File: {'✓' if os.path.exists(receptor_gpf) else '✗'} {receptor_gpf}")

## 2. Ligand Preparation with Explicit Water Molecules

Prepare the ligand by adding hydrogens and explicit water molecules (W atoms):

In [ ]:
# 2. Ligand Preparation with Explicit Waters

# Step 2a: Add hydrogens with scrub.py (no -w flag yet)
ligand_h_sdf = f"{results_dir}/{pdb_id}_ligandH.sdf"

scrub_cmd = [
    str(scrub),
    ligand_sdf,
    "-o", ligand_h_sdf
]

print("Adding hydrogens to ligand...")
print(f"Command: {' '.join(scrub_cmd)}")

result = subprocess.run(scrub_cmd, capture_output=True, text=True)
print("\nSTDOUT:")
print(result.stdout)
if result.stderr:
    print("\nSTDERR:")
    print(result.stderr)

# Step 2b: Convert to PDBQT with explicit water molecules (-w flag)
ligand_pdbqt = f"{results_dir}/{pdb_id}_ligand.pdbqt"

prepare_ligand_cmd = [
    str(mk_prepare_ligand),
    "-i", ligand_h_sdf,
    "-o", ligand_pdbqt,
    "-w"  # Add explicit water molecules
]

print("\nConverting to PDBQT with explicit water molecules...")
print(f"Command: {' '.join(prepare_ligand_cmd)}")

result = subprocess.run(prepare_ligand_cmd, capture_output=True, text=True)
print("\nSTDOUT:")
print(result.stdout)
if result.stderr:
    print("\nSTDERR:")
    print(result.stderr)

print(f"\nGenerated files:")
print(f"  Ligand with H: {'✓' if os.path.exists(ligand_h_sdf) else '✗'} {ligand_h_sdf}")
print(f"  Ligand PDBQT with W: {'✓' if os.path.exists(ligand_pdbqt) else '✗'} {ligand_pdbqt}")

# Count water molecules added
if os.path.exists(ligand_pdbqt):
    with open(ligand_pdbqt, 'r') as f:
        water_count = sum(1 for line in f if 'W' in line.split()[-1:] and ('ATOM' in line or 'HETATM' in line))
    print(f"  Water molecules added: {water_count}")

## 3. Generate Affinity Maps with AutoGrid4

Generate the affinity maps using AutoGrid4 and create the water affinity map:

In [ ]:
# 3. Generate Affinity Maps

# Step 3a: Generate affinity maps with AutoGrid4
grid_log = f"{receptor_output}.glg"

autogrid_cmd = [
    "autogrid4",
    "-p", receptor_gpf,
    "-l", grid_log
]

print("Generating affinity maps with AutoGrid4...")
print(f"Command: {' '.join(autogrid_cmd)}")

# Change to results directory for map generation
original_cwd = os.getcwd()
os.chdir(results_dir)

try:
    result = subprocess.run(autogrid_cmd, capture_output=True, text=True, cwd=results_dir)
    print("\nSTDOUT:")
    print(result.stdout)
    if result.stderr:
        print("\nSTDERR:")
        print(result.stderr)
finally:
    os.chdir(original_cwd)

# Step 3b: Generate water affinity map with mapwater.py
water_map = f"{results_dir}/{receptor_base}.W.map"

mapwater_cmd = [
    str(mapwater),
    "-r", receptor_pdbqt,
    "-s", water_map
]

print("\nGenerating water affinity map...")
print(f"Command: {' '.join(mapwater_cmd)}")

result = subprocess.run(mapwater_cmd, capture_output=True, text=True)
print("\nSTDOUT:")
print(result.stdout)
if result.stderr:
    print("\nSTDERR:")
    print(result.stderr)

# Check generated map files
maps_fld = f"{results_dir}/{receptor_base}.maps.fld"
oa_map = f"{results_dir}/{receptor_base}.OA.map"
hd_map = f"{results_dir}/{receptor_base}.HD.map"

print(f"\nGenerated map files:")
print(f"  Maps field file: {'✓' if os.path.exists(maps_fld) else '✗'} {maps_fld}")
print(f"  OA map: {'✓' if os.path.exists(oa_map) else '✗'} {oa_map}")
print(f"  HD map: {'✓' if os.path.exists(hd_map) else '✗'} {hd_map}")
print(f"  Water map: {'✓' if os.path.exists(water_map) else '✗'} {water_map}")

## 4. Hydrated Docking with AutoDock4 Forcefield

Perform the docking using Vina with AutoDock4 forcefield (required for hydrated docking):

In [ ]:
# 4. Hydrated Docking with AutoDock4 Forcefield

output_pdbqt = f"{results_dir}/{pdb_id}_{ligand_name}_ad4_out.pdbqt"
maps_prefix = f"{results_dir}/{receptor_base}"

# Vina command with AutoDock4 forcefield
vina_cmd = [
    "vina",
    "--ligand", ligand_pdbqt,
    "--maps", maps_prefix,
    "--scoring", "ad4",  # Use AutoDock4 forcefield
    "--exhaustiveness", "32",
    "--out", output_pdbqt
]

print("Running hydrated docking with AutoDock4 forcefield...")
print(f"Command: {' '.join(vina_cmd)}")
print("\nThis may take several minutes...")

result = subprocess.run(vina_cmd, capture_output=True, text=True)
print("\nSTDOUT:")
print(result.stdout)
if result.stderr:
    print("\nSTDERR:")
    print(result.stderr)

print(f"\nDocking completed:")
print(f"  Output file: {'✓' if os.path.exists(output_pdbqt) else '✗'} {output_pdbqt}")

# Parse docking results
if os.path.exists(output_pdbqt):
    with open(output_pdbqt, 'r') as f:
        lines = f.readlines()

    print("\nDocking Results Summary:")
    for line in lines:
        if line.startswith("REMARK VINA RESULT:"):
            parts = line.split()
            if len(parts) >= 4:
                energy = parts[3]
                rmsd1 = parts[4] if len(parts) > 4 else "N/A"
                rmsd2 = parts[5] if len(parts) > 5 else "N/A"
                print(f"  Mode: {parts[2]}, Energy: {energy} kcal/mol, RMSD: {rmsd1}, {rmsd2}")

## 5. Post-processing with Water Analysis

Process the docking results to filter and score water molecules:

In [ ]:
# 5. Post-processing with dry.py

# Use dry.py to filter and score water molecules
dry_cmd = [
    str(dry),
    "-r", receptor_pdbqt,
    "-m", water_map,
    "-i", output_pdbqt
]

print("Processing results with water analysis...")
print(f"Command: {' '.join(dry_cmd)}")

result = subprocess.run(dry_cmd, capture_output=True, text=True)
print("\nSTDOUT:")
print(result.stdout)
if result.stderr:
    print("\nSTDERR:")
    print(result.stderr)

# Check for generated output files
dry_scored_pdbqt = f"{results_dir}/{pdb_id}_{ligand_name}_ad4_out_DRY_SCORED.pdbqt"

print(f"\nPost-processing results:")
print(f"  Scored output: {'✓' if os.path.exists(dry_scored_pdbqt) else '✗'} {dry_scored_pdbqt}")

# Analyze water scoring results
if result.stdout:
    lines = result.stdout.split('\n')
    strong_waters = []
    weak_waters = []
    displaced_waters = []

    for line in lines:
        if "Water STRONG" in line:
            strong_waters.append(line.strip())
        elif "Water  WEAK" in line:
            weak_waters.append(line.strip())
        elif "Water DISPLC" in line:
            displaced_waters.append(line.strip())

    print(f"\nWater Analysis Results:")
    print(f"  Strong waters (conserved): {len(strong_waters)}")
    print(f"  Weak waters: {len(weak_waters)}")
    print(f"  Displaced waters: {len(displaced_waters)}")

    if strong_waters:
        print("\nStrong Waters (first 5):")
        for water in strong_waters[:5]:
            print(f"    {water}")

    if weak_waters:
        print("\nWeak Waters (first 5):")
        for water in weak_waters[:5]:
            print(f"    {water}")

## 6. Results Visualization

Visualize the docking results with conserved water molecules:

In [ ]:
# 6. Visualization of Hydrated Docking Results

def Hydrated3DView(receptorPDB=None, ligandPDBQT=None, scoredPDBQT=None, width=800, height=600):
    """
    3D visualization function for hydrated docking results
    """
    view = py3Dmol.view(width=width, height=height)

    # Add receptor
    if receptorPDB and os.path.exists(receptorPDB):
        with open(receptorPDB, 'r') as f:
            receptor_data = f.read()
        view.addModel(receptor_data, 'pdb')
        view.addStyle({'model': 0}, {'cartoon': {'color': 'lightgray', 'opacity': 0.8}})

    # Add original ligand result
    if ligandPDBQT and os.path.exists(ligandPDBQT):
        with open(ligandPDBQT, 'r') as f:
            # Read only the first model
            ligand_lines = []
            in_model = False
            for line in f:
                if line.startswith('MODEL 1') or (not in_model and line.startswith('ATOM')):
                    in_model = True
                if in_model:
                    ligand_lines.append(line)
                    if line.startswith('ENDMDL'):
                        break

        if ligand_lines:
            ligand_data = ''.join(ligand_lines)
            view.addModel(ligand_data, 'pdbqt')
            # Ligand atoms (non-water)
            view.addStyle({'model': 1, 'atom': '!W'}, {'stick': {'colorscheme': 'greenCarbon', 'radius': 0.2}})
            # Water molecules from original docking
            view.addStyle({'model': 1, 'atom': 'W'}, {'sphere': {'color': 'blue', 'radius': 0.3, 'opacity': 0.7}})

    # Add scored results if available
    if scoredPDBQT and os.path.exists(scoredPDBQT):
        with open(scoredPDBQT, 'r') as f:
            scored_data = f.read()
        view.addModel(scored_data, 'pdbqt')
        # Strong waters (red)
        view.addStyle({'model': 2, 'resi': 'WAT'}, {'sphere': {'color': 'red', 'radius': 0.4}})

    view.zoomTo()
    return view

# Create visualization
receptor_pdb_path = receptor_pdb if os.path.exists(receptor_pdb) else None
view = Hydrated3DView(
    receptorPDB=receptor_pdb_path,
    ligandPDBQT=output_pdbqt,
    scoredPDBQT=dry_scored_pdbqt if os.path.exists(dry_scored_pdbqt) else None
)

print("3D Visualization:")
print("- Gray: Receptor protein")
print("- Green sticks: Ligand")
print("- Blue spheres: All water molecules")
print("- Red spheres: Conserved/strong water molecules")

view.show()

## 7. Results Summary and Analysis

Summary of the hydrated docking experiment:

In [ ]:
# 7. Results Summary

print("=" * 60)
print(f"HYDRATED DOCKING RESULTS SUMMARY")
print("=" * 60)

print(f"Experiment: {experiment_date}_Hydrated_{pdb_id.upper()}_{ligand_name}-{target_name}")
print(f"Target: {target_name} (PDB: {pdb_id.upper()})")
print(f"Ligand: {ligand_name}")
print(f"Method: Hydrated docking with AutoDock4 forcefield")

print(f"\nFiles Generated:")
print(f"  Results directory: {results_dir}")

# List all generated files
if os.path.exists(results_dir):
    files = os.listdir(results_dir)
    files.sort()

    categories = {
        'Input preparation': ['.sdf', '_ligandH.sdf'],
        'Receptor files': ['.pdbqt', '.gpf', '.glg'],
        'Map files': ['.map', '.fld'],
        'Docking results': ['_ad4_out.pdbqt'],
        'Processed results': ['_DRY_SCORED.pdbqt']
    }

    for category, extensions in categories.items():
        print(f"\n  {category}:")
        category_files = [f for f in files if any(ext in f for ext in extensions)]
        for file in category_files:
            file_path = os.path.join(results_dir, file)
            size = os.path.getsize(file_path) if os.path.exists(file_path) else 0
            print(f"    - {file} ({size:,} bytes)")

# Performance metrics
if os.path.exists(output_pdbqt):
    print(f"\nDocking Performance:")

    with open(output_pdbqt, 'r') as f:
        content = f.read()

    # Count poses
    pose_count = content.count('MODEL')
    print(f"  Total poses generated: {pose_count}")

    # Extract best energy
    lines = content.split('\n')
    for line in lines:
        if 'REMARK VINA RESULT:' in line and 'MODEL 1' in line:
            parts = line.split()
            if len(parts) >= 4:
                best_energy = parts[3]
                print(f"  Best binding energy: {best_energy} kcal/mol")
            break

print(f"\nNext Steps:")
print("1. Analyze water molecule conservation patterns")
print("2. Compare with experimental water positions (if available)")
print("3. Evaluate structural accuracy vs. rigid/flexible docking")
print("4. Consider water-mediated interactions in structure-activity relationships")

print(f"\n✓ Hydrated docking experiment completed successfully!")